## Cartoonify faces

Import library

In [1]:
import cv2
import dlib
import numpy as np
import os

Collect HAAR Cascade filters

In [2]:
cascPathface = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
cascPatheyes = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_eye_tree_eyeglasses.xml"

Create Cascade Classifiers
- Face
- Eyes

In [3]:
faceCascade = cv2.CascadeClassifier(cascPathface)
eyeCascade = cv2.CascadeClassifier(cascPatheyes)

Capture Video feed from webcam

In [4]:
video_capture = cv2.VideoCapture(0)

Read frame from videocam

In [5]:


# Check if the webcam is opened
if not video_capture.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = video_capture.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break
    cv2.imshow('Face Video', frame)
    if cv2.waitKey(1) & 0xFF == 27:  # Exit on 'Esc' key
        break



Convert the frame to GrayScale

In [6]:
while True:
    ret, frame = video_capture.read()
    if not ret:
        print("Failed to capture frame. Check your camera connection or source.")
        break
    cv2.imshow('Face Video', frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('Gray Video', gray)
    if cv2.waitKey(1) & 0xFF == 27:  # Exit on 'Esc' key
        break


Detect faces

In [7]:
faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE
)

In [8]:
faces

array([[314, 162, 228, 228]])

Initialize webcam and face detector

In [9]:

cap = cv2.VideoCapture(0)  # Use 0 for default webcam
detector = dlib.get_frontal_face_detector()

Load the cartoon mask

In [10]:

mask_path = "C:/Users/devan/Downloads/Mini-removebg-preview.png"
mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)  # Ensure the mask has an alpha channel

Capture Cartoonify face

In [11]:
# Check if the mask has an alpha channel
if mask.shape[2] == 3:  # If the mask has only RGB channels
    print("Adding alpha channel to the mask")
    alpha_channel = np.ones((mask.shape[0], mask.shape[1]), dtype=mask.dtype) * 255  # Fully opaque
    mask = np.dstack((mask, alpha_channel))  # Add alpha channel

print("Press 'q' to quit the webcam stream.")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Detect faces
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    for face in faces:
        # Get face coordinates
        x, y, w, h = face.left(), face.top(), face.width(), face.height()

        # Resize the mask to the face size
        resized_mask = cv2.resize(mask, (w, h))

        # Overlay the mask on the face
        for i in range(h):
            for j in range(w):
                if resized_mask[i, j, 3] > 0:  # Alpha channel check
                    # Ensure indices are within bounds
                    if 0 <= y + i < frame.shape[0] and 0 <= x + j < frame.shape[1]:
                        frame[y + i, x + j] = resized_mask[i, j, :3]  # Copy RGB channels
    
    # Display the frame with the cartoon mask
    cv2.imshow("Cartoon Mask Webcam", frame)
    
    # Break on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

Press 'q' to quit the webcam stream.


In [12]:
# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Define the codec
out = cv2.VideoWriter('C:/Users/devan/Downloads/cartoon_mask_output.mp3', fourcc, 20.0, (frame.shape[1], frame.shape[0]))

# Save images to a custom folder
cv2.imwrite(f"C:/Users/devan/Downloads/cartoon_mask_output.png", frame)


True

In [13]:
# Write the processed frame to the video file
out.write(frame)


In [14]:
# Release webcam and close window
cap.release()
cv2.destroyAllWindows()# Check if the mask has an alpha channel